In [1]:
from skimage.filters import gabor_kernel
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt 
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import math

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train = x_train.astype('float32') / 255.0
# x_test = x_test.astype('float32') / 255.0

# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100
IMAGE_SIZE = 32
NUM_OF_CLASSES = 10

def process_dataset(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
#     image = tf.image.per_image_standardization(image)
    
#     image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize images from 32x32 to 227x227
#     image = tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))

#     image = image/255.0
         
    label = tf.one_hot(tf.cast(label, tf.int32), NUM_OF_CLASSES)
        
    return image, label

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

def construct_dataset(ds):
    ds = ds.shuffle(buffer_size=BATCH_SIZE)
    
#     ds=ds.repeat()
    ds = ds.batch(BATCH_SIZE, drop_remainder=False)
#     ds = ds.map(process_dataset,num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_data = construct_dataset(train_data)
test_data = construct_dataset(test_data)

steps_per_epoch=math.ceil(50000/BATCH_SIZE)
validation_steps=math.ceil(10000/BATCH_SIZE)


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


In [3]:
# Importing the Keras libraries and packages
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam

# dimensionality of input and latent encoded representations
inpt_dim = (32, 32, 3)

inpt_img = Input(shape=inpt_dim)

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape = inpt_dim )(inpt_img)

# Block 1
cl1 = Conv2D(64, (3, 3), strides=(2, 2),activation='relu')(rescale)
bnl2 = BatchNormalization()(cl1)
# afl3 = Activation('relu')(bnl2)
pl4 = MaxPooling2D(pool_size = (2, 2))(bnl2)

# Adding a second convolutional layer
cl5 = Conv2D(128, (3, 3), strides=(1, 1), activation='relu')(pl4)
bnl6 = BatchNormalization()(cl5)
# afl7 = Activation('relu')(bnl6)
pl8 = MaxPooling2D(pool_size = (2, 2))(bnl6)
bnl9 = BatchNormalization()(pl8)

# Step 3 - Flattening
fl10 = Flatten()(bnl9)

# Step 4 - Full connection
dol11 = Dropout(0.5)(fl10)
dl12 = Dense(units = 256, activation = 'relu')(dol11)
dol13 = Dropout(0.2)(dl12)
dl14 = Dense(units = 64, activation = 'relu')(dol13)
dol15 = Dropout(0.1)(dl14)
output = Dense(units = 10, activation = 'softmax')(dol15)

classifier = Model(inpt_img, output)

# Compiling the CNN
opt = RMSprop(learning_rate=0.001)
# opt = Adam(learning_rate=0.01)

classifier.compile(optimizer = opt, loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                   metrics = ['accuracy'])

print(classifier.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 15, 15, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 128)         73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 128)        

In [ ]:
# Fitting the CNN to the images

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                              min_delta=1e-4, mode='min', verbose=1)

stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                         restore_best_weights=True, verbose=1)



hist = classifier.fit(train_data,  epochs=1000, 
                   callbacks=[stop_alg, reduce_lr], 
                      validation_steps=validation_steps,
                      steps_per_epoch=steps_per_epoch,
                   validation_data=test_data)



Epoch 1/1000
500/500 [==============================] - 40s 80ms/step - loss: 2.0749 - accuracy: 0.3793 - val_loss: 2.2210 - val_accuracy: 0.2247
Epoch 2/1000
500/500 [==============================] - 42s 84ms/step - loss: 2.0023 - accuracy: 0.4529 - val_loss: 2.1252 - val_accuracy: 0.3286
Epoch 3/1000
500/500 [==============================] - 45s 89ms/step - loss: 1.9763 - accuracy: 0.4808 - val_loss: 2.0623 - val_accuracy: 0.3942
Epoch 4/1000
500/500 [==============================] - 45s 90ms/step - loss: 1.9582 - accuracy: 0.4992 - val_loss: 2.0025 - val_accuracy: 0.4546
Epoch 5/1000
500/500 [==============================] - 48s 97ms/step - loss: 1.9525 - accuracy: 0.5053 - val_loss: 2.1389 - val_accuracy: 0.3176
Epoch 6/1000
500/500 [==============================] - 48s 97ms/step - loss: 1.9426 - accuracy: 0.5156 - val_loss: 1.9708 - val_accuracy: 0.4870
Epoch 7/1000
500/500 [==============================] - 48s 96ms/step - loss: 1.9391 - accuracy: 0.5190 - val_loss: 1.9738 -

500/500 [==============================] - 63s 126ms/step - loss: 1.8117 - accuracy: 0.6484 - val_loss: 1.8231 - val_accuracy: 0.6357
Epoch 55/1000
500/500 [==============================] - 61s 123ms/step - loss: 1.8139 - accuracy: 0.6451 - val_loss: 1.8128 - val_accuracy: 0.6467
Epoch 56/1000
500/500 [==============================] - 63s 127ms/step - loss: 1.8087 - accuracy: 0.6512 - val_loss: 1.8175 - val_accuracy: 0.6420
Epoch 57/1000
500/500 [==============================] - 60s 121ms/step - loss: 1.8100 - accuracy: 0.6498 - val_loss: 1.8102 - val_accuracy: 0.6488
Epoch 58/1000
500/500 [==============================] - 59s 119ms/step - loss: 1.8099 - accuracy: 0.6496 - val_loss: 1.8254 - val_accuracy: 0.6337
Epoch 59/1000
500/500 [==============================] - 59s 119ms/step - loss: 1.8072 - accuracy: 0.6525 - val_loss: 1.8172 - val_accuracy: 0.6428
Epoch 60/1000
500/500 [==============================] - 61s 123ms/step - loss: 1.8077 - accuracy: 0.6519 - val_loss: 1.8355 -

500/500 [==============================] - ETA: 0s - loss: 1.7772 - accuracy: 0.6822
Epoch 00108: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
500/500 [==============================] - 59s 119ms/step - loss: 1.7772 - accuracy: 0.6822 - val_loss: 1.7964 - val_accuracy: 0.6627
Epoch 109/1000
500/500 [==============================] - 59s 118ms/step - loss: 1.7745 - accuracy: 0.6854 - val_loss: 1.7781 - val_accuracy: 0.6811
Epoch 110/1000
500/500 [==============================] - 59s 119ms/step - loss: 1.7722 - accuracy: 0.6874 - val_loss: 1.7768 - val_accuracy: 0.6839
Epoch 111/1000
500/500 [==============================] - 59s 118ms/step - loss: 1.7755 - accuracy: 0.6839 - val_loss: 1.7938 - val_accuracy: 0.6651
Epoch 112/1000
500/500 [==============================] - 60s 120ms/step - loss: 1.7733 - accuracy: 0.6863 - val_loss: 1.7819 - val_accuracy: 0.6782
Epoch 113/1000
500/500 [==============================] - 61s 123ms/step - loss: 1.7727 - accuracy: 0.6872

Epoch 160/1000
201/500 [===========>..................] - ETA: 26s - loss: 1.7660 - accuracy: 0.6937